<a href="https://colab.research.google.com/github/limseo12/YOLOv8_Object_Detection/blob/main/custom_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Vo3weMCC3nS9jZ3v6tEV")
project = rf.workspace("philip-hawkins-zadu0").project("automotivefeatures")
dataset = project.version(1).download("yolov8")

In [ ]:
!cat /content/AutomotiveFeatures-1/data.yaml

In [ ]:
!pip install PyYAML

In [ ]:
#경로설정 직접경로 설정을해서 학습을 시도하면 오류가난다... 기존에 있던 yaml 데이터 그대로 학습을 시도하면 오류가 나지 않는다.
import yaml

Auto_data = {
        'train': '/content/AutomotiveFeatures-1/train/images',
        'test': '/content/AutomotiveFeatures-1/test/images',
        'valid': '/content/AutomotiveFeatures-1/valid/images',
        'names' : ['EntireVehicle','Interior'],
        'nc' : 2 }

with open('/content/AutomotiveFeatures-1/Auto_data.yaml', 'w') as f:
  yaml.dump(Auto_data, f)

with open('/content/AutomotiveFeatures-1/Auto_data.yaml', 'r') as f:
  Auto_data = yaml.safe_load(f)
  display(Auto_data)

In [ ]:
pwd

In [ ]:
%cd /content/AutomotiveFeatures-1

In [ ]:
!cat /content/AutomotiveFeatures-1/Auto_data.yaml

## YOLOv8 설치

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics

ultralytics.checks()

## Train

In [ ]:
#Load a pre-trained model
from ultralytics import YOLO
model_x = YOLO('yolov8x.pt')

In [ ]:
print(type(model_x.names), len(model_x.names))
print(model_x.names)

In [ ]:
model_x.train(data= '/content/AutomotiveFeatures-1/data.yaml', epochs=10, patience=30, batch=32, imgsz=316)

In [ ]:
results = model_x.predict(source='/content/AutomotiveFeatures-1/test/images', save = True)

In [ ]:
print(type(results), len(results))

In [ ]:
import numpy as np

for result in results:

    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print('\n{class num:counts} =', uniq_cnt_dict,'\n')

    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model_x.names[int(c)])

In [ ]:
#첫번째
from PIL import Image

with Image.open('/content/AutomotiveFeatures-1/runs/detect/predict/416458491_Clipped_mp4-1_jpg.rf.99b190fae06ec6c7772a7bf2d997b22c.jpg') as pred_image:

  pred_image.show()

In [ ]:
#네번째
from PIL import Image

with Image.open('/content/AutomotiveFeatures-1/runs/detect/predict/7776537_1218102_mp4-23_jpg.rf.c856ec124ede75dfa694f932b81e52c8.jpg') as pred_image:

  pred_image.show()

## 결과 확인 및 다운

In [ ]:
import glob

detetced_image_list = glob.glob(('/content/runs/detect/predict/*'))

detected_image_nums = len(detetced_image_list)

print(detected_image_nums)

print(detetced_image_list)

In [ ]:
# 다운로드를 위한 inference image 압축

import zipfile
import os

if not os.path.exists('/content/detected_result/'):
    os.mkdir('/content/detected_result/')
    print('detected_result dir is created !!!')


with zipfile.ZipFile('/content/detected_result/detected_images.zip', 'w') as detected_images:

    for idx in range(detected_image_nums):
        detected_images.write(detetced_image_list[idx])